In [2]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import sklearn

## load data and drop small targets 

In [6]:
kyotp_int = pd.read_csv("preparedData03-06_15Int.csv",index_col=[0])
kyotp_int.head()

,Duration,Source bytes,Destination bytes,Count,Same srv rate,Serror rate,Srv serror rate,Dst host count,Dst host srv count,Dst host same src port rate,Dst host serror rate,Dst host srv serror rate,0,0.1,0.2,Label
0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,3,0
1,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,3,0
2,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,3,0
3,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,1,3,0
4,0.000345,1.970562e-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,2,3,0


In [14]:
kyotp_int = kyotp_int.drop_duplicates()

In [15]:
X = kyotp_int.drop("Label",axis = 1)
y =  kyotp_int.Label

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [17]:
X_train.head()

,Duration,Source bytes,Destination bytes,Count,Same srv rate,Serror rate,Srv serror rate,Dst host count,Dst host srv count,Dst host same src port rate,Dst host serror rate,Dst host srv serror rate,0,0.1,0.2
3463707,1.328432e-05,2.561731e-07,1.309427e-06,0.02,1.0,0.0,0.00,0.29,0.29,0.0,0.0,0.0,10,1,8
514063,4.103422e-09,1.724242e-08,2.632273e-08,0.00,0.0,0.0,0.00,0.95,0.97,0.0,0.0,0.0,10,2,1
8936446,4.232387e-09,3.891861e-08,6.916168e-08,0.74,1.0,0.0,0.01,0.99,0.99,0.0,0.0,0.0,10,2,1
500171,4.982727e-09,2.167619e-08,3.870989e-08,0.07,1.0,0.0,0.12,0.97,0.97,0.0,0.0,0.0,10,2,1
8781566,8.270037e-07,4.926406e-08,2.363884e-07,0.00,0.0,0.0,0.00,0.46,0.50,1.0,0.0,0.0,10,2,1


In [20]:
print(str(sum(y_test)) + " / " + str(len(y_test)))

154165 / 1648171


In [21]:
print(str(sum(y_train)) + " / " + str(len(y_train)))

313558 / 3346286


In [22]:
kyotp_int.shape

(4994457, 16)

## train

In [19]:
from sklearn.ensemble import HistGradientBoostingClassifier
clfhg = HistGradientBoostingClassifier(learning_rate = 0.02, max_depth = 20, max_iter = 600,max_leaf_nodes = 31, min_samples_leaf = 15, categorical_features=[12,13,14])
clfhg.fit(X_train, y_train)
y_pred = clfhg.predict(X_test)
from sklearn.metrics import precision_score
print("score: " + str(precision_score(y_test, y_pred)))
from sklearn.metrics import recall_score
print("score: " + str(recall_score(y_test, y_pred)))
from sklearn.metrics import roc_auc_score
print("score: " + str(roc_auc_score(y_test, y_pred)))
from sklearn.metrics import f1_score
print("score: " + str(f1_score(y_test, y_pred)))

score: 0.9682861535199322
score: 0.9478545714007719
score: 0.9723255518385407
score: 0.9579614326874856


In [22]:
#{'learning_rate': 0.03, 'max_depth': 20, 'max_iter': 600, 'max_leaf_nodes': 31, 'min_samples_leaf': 15}
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

parameters = [{'learning_rate':[0.1,0.5,0.003],'max_iter':[100,200],'max_depth' : [10,20,50],'min_samples_leaf':[10,20,30], 'max_leaf_nodes':[15,31,63] }]
parameters = [{'learning_rate':[0.5,0.003],'max_iter':[100,200]}]
parameters = [{'learning_rate':[0.03],'max_iter':[800,600],'max_depth' : [30,20,40],'min_samples_leaf':[10,15], 'max_leaf_nodes':[31] }]
scores = ['f1']
# {'learning_rate': 0.03, 'max_depth': 50, 'max_iter': 400, 'max_leaf_nodes': 31, 'min_samples_leaf': 10}
#{'learning_rate': 0.03, 'max_depth': 40, 'max_iter': 600, 'max_leaf_nodes': 31, 'min_samples_leaf': 10}
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(HistGradientBoostingClassifier(), parameters, scoring=score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    f = open("HistGradientBoostingClassifierParamsearch.txt", "a")
   
    
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        f.write("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()
    f.close()
    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()
    f = open("HistGradientBoostingClassifierParamsearch.txt", "a")
    f.write("classification_report(y_true, y_pred)")
    f.close()

# Tuning hyper-parameters for f1

Best parameters set found on development set:

{'learning_rate': 0.03, 'max_depth': 20, 'max_iter': 600, 'max_leaf_nodes': 31, 'min_samples_leaf': 15}

Grid scores on development set:

0.817 (+/-0.004) for {'learning_rate': 0.03, 'max_depth': 30, 'max_iter': 800, 'max_leaf_nodes': 31, 'min_samples_leaf': 10}
0.817 (+/-0.004) for {'learning_rate': 0.03, 'max_depth': 30, 'max_iter': 800, 'max_leaf_nodes': 31, 'min_samples_leaf': 15}
0.817 (+/-0.004) for {'learning_rate': 0.03, 'max_depth': 30, 'max_iter': 600, 'max_leaf_nodes': 31, 'min_samples_leaf': 10}
0.817 (+/-0.004) for {'learning_rate': 0.03, 'max_depth': 30, 'max_iter': 600, 'max_leaf_nodes': 31, 'min_samples_leaf': 15}
0.817 (+/-0.004) for {'learning_rate': 0.03, 'max_depth': 20, 'max_iter': 800, 'max_leaf_nodes': 31, 'min_samples_leaf': 10}
0.817 (+/-0.004) for {'learning_rate': 0.03, 'max_depth': 20, 'max_iter': 800, 'max_leaf_nodes': 31, 'min_samples_leaf': 15}
0.817 (+/-0.004) for {'learning

In [1]:
f = open("demofile2.txt", "a")
f.write("Now the file has more content!")
f.close()

In [ ]:
y_pred = clf.predict(X_test)
from sklearn.metrics import precision_score
print("score: " + str(precision_score(y_test, y_pred)))

In [ ]:
y_pred = clf.predict(X_test)
from sklearn.metrics import recall_score
print("score: " + str(recall_score(y_test, y_pred)))

In [ ]:
np.sum([y_test])

In [ ]:
np.sum([y_pred])

In [ ]:
y_pred = clf.predict(X_test[:50])

In [ ]:
y_pred

In [ ]:
y_test[:50]

In [ ]:
import matplotlib.pyplot as plt
importances = clf.feature_importances_
forest_importances = pd.Series(importances, index = X.columns)

fig, ax = plt.subplots()
forest_importances.plot.bar()
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()